In [1]:
import geemap
import ee

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:
geemap.extract_values_to_points()

In [ ]:
# 导入需要提取的图像
MOD13 = ee.ImageCollection('MODIS/006/MOD13A2')\
    .filterDate('2018-01', '2018-02').first().select('NDVI')

Map.addLayer(MOD13, {}, 'MOD13')

In [ ]:
# 在Map上建立点集
temp_point = ee.FeatureCollection(Map.draw_features)
temp_point.getInfo()

In [ ]:
# 点值提取
temp = MOD13.sampleRegions(temp_point, geometries=True)
temp.getInfo()

In [ ]:
# 将要提取的值设置为一个属性，并填充为-9999
def setNDVI(feature):
    return feature.set('NDVI', -9999)
temp_point1 = temp_point.map(setNDVI)
temp_point1.getInfo()

In [ ]:
# 修改点值提取后的要素集的id
temp1 = ee.List([])
def setID1(feature, List):
    feature1 = feature.set('system:index', ee.String(feature.get('system:index')).split('_').get(0))
    return ee.List(List).add(feature1)
temp2 = ee.FeatureCollection(ee.List(temp.iterate(setID1, temp1)))
temp2.getInfo()

In [ ]:
# 筛选空值
filter = ee.Filter.equals(
  leftField='system:index',
  rightField='system:index'
)
simpleJoin = ee.Join.inverted()
simpleJoined = simpleJoin.apply(temp_point1, temp2, filter)

simpleJoined.getInfo()

In [ ]:
# 合并要素集
temp3 = temp2.merge(simpleJoined)
temp3.getInfo()

In [ ]:
temp4 = ee.List([])
def setID2(feature, List):
    feature1 = feature.set('system:index', ee.String(feature.get('system:index')).split('_').get(-1))
    feature1 = feature1.set('lon', ee.List(feature.geometry().coordinates().get(0)))
    feature1 = feature1.set('lat', ee.List(feature.geometry().coordinates().get(1)))
    return ee.List(List).add(feature1)
temp5 = ee.FeatureCollection(ee.List(temp3.iterate(setID2, temp1))).sort('system:index')
temp5.getInfo()

In [ ]:
geemap.ee_to_shp(temp5, 'temp2.shp')

In [ ]:
ee.List(temp5.first().geometry().coordinates()).get(0).getInfo()

In [ ]:
temp5.geometry().coordinates().getInfo()